In [1]:
from tagger.models.received_item import ReceivedItem, ReceivedItemStatus
from tagger.core.mongo.models.order import Order, OrderStatus
from tagger.viewsets.order.change_status import ChangeStatusSerializer, _change_status
from django.contrib.auth.models import User
from tagger.models.inventory import Inventory, InventoryStatus
from tagger.models import ShippingNotice, Package, Courier
from tagger.models.package import PackageStatus

import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
import pandas as pd

courier = Courier.objects.get(name="CJ대한통운")

In [3]:
with open("../Downloads/확장주문검색_20211118_163649.xlsx", "rb") as f:
    df = pd.read_excel(f, dtype=str)
    df.dropna(subset=["주문번호"], inplace=True)
    for _, row in df.iterrows():
        print("--------------------------------------------------------------------------------")
        package_code = row["주문번호"]
        package = Package.objects.get(code=package_code)
        print(row["주문번호"], row["판매처 상품명"], row["수령자이름"])
        for item in package.shipping_notice_items.all():
            print(f"""ㄴ ORDER {item.item.extended_order.code} ({item.item.extended_order.order.payment.buyername}) : ITEM {item.item.product_code}""")
len(df)

--------------------------------------------------------------------------------
PKG-211118-EK 글리터 구스 점퍼 홍은미
ㄴ ORDER TNPBY (홍은미) : ITEM SJSJ___글리터구스점퍼___76(44)
--------------------------------------------------------------------------------
PKG-211118-6N 오버사이즈 소피 다운 점퍼(브라운) 송효은
ㄴ ORDER AQRE4 (송효은) : ITEM BEANPOLEL___오버사이즈소피다운점퍼(브라운)___95
--------------------------------------------------------------------------------
PKG-211118-66 벨티드 플랩 코트(카멜) 김혜연
ㄴ ORDER 6MM7D (김혜연) : ITEM CCCOLLECT___벨티드플랩코트(카멜)___55
--------------------------------------------------------------------------------
PKG-211118-E8 디테쳐블 플리츠 후드 드레스(SEPIA BEIGE) 안나
ㄴ ORDER MDVK7 (안나) : ITEM MINE___디테쳐블플리츠후드드레스(SEPIABEIGE)___82(55)
--------------------------------------------------------------------------------
PKG-211118-4U [빈폴스포츠]푸퍼 미드 다운 점퍼(네이비) 하다정
ㄴ ORDER QWK9J (하다정) : ITEM BEANPOLEL___[빈폴스포츠]푸퍼미드다운점퍼(네이비)___95
--------------------------------------------------------------------------------
PKG-211118-4K [빈폴스포츠] (MEN) 

15

In [4]:
for _, row in df.iterrows():
    package = Package.objects.get(code=row["주문번호"])
    package.courier = courier
    package.tracking_number = row['송장번호']
    package.status = PackageStatus.SHIPPED
    package.save()
    
    for eo in package.extended_orders:
        order = eo.order
        status_change_data = {
            "status": OrderStatus.DELIVERY_STARTED,
            "delivery_tracking_number": package.courier.name + " " + package.tracking_number,
            "delivery_tracking_url": package.tracking_url,
        }
        s = ChangeStatusSerializer(data=status_change_data, instance=order)
        s.is_valid(raise_exception=True)
        _change_status(s, order, User.objects.get(username="backoffice_server"), id=str(order.id))
        print("-----------------------------------------------------------------------")
        print(f"""[{package.code}] {package.recipient_name} ({package.recipient_mobile})""")
        for item in package.shipping_notice_items.all():
            print(f"ㄴ https://office.alloff.co/orders/{item.item.extended_order.order.code}")
            print(f"""   [INV {item.inventory.code}] {item.item.name}""")
print("-----------------------------------------------------------------------")        

/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField Package.shipped_at received a naive datetime (2021-11-18 07:37:53.949553) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


-----------------------------------------------------------------------
[PKG-211118-EK] 홍은미 (01031903669)
ㄴ https://office.alloff.co/orders/TNPBY
   [INV 211118-009] 글리터 구스 점퍼 
-----------------------------------------------------------------------
[PKG-211118-6N] 송효은 (01055001835)


/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField Package.shipped_at received a naive datetime (2021-11-18 07:37:54.386494) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField Package.shipped_at received a naive datetime (2021-11-18 07:37:54.581772) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


ㄴ https://office.alloff.co/orders/AQRE4
   [INV 211118-017] 오버사이즈 소피 다운 점퍼(브라운) 
-----------------------------------------------------------------------
[PKG-211118-66] 김혜연 (01066415346)
ㄴ https://office.alloff.co/orders/6MM7D
   [INV 211118-011] 벨티드 플랩 코트(카멜) 


/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField Package.shipped_at received a naive datetime (2021-11-18 07:37:54.771419) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField Package.shipped_at received a naive datetime (2021-11-18 07:37:54.962623) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


...

-----------------------------------------------------------------------
[PKG-211118-HY] 유소라 (01092145711)
ㄴ https://office.alloff.co/orders/HWP8C
   [INV 211118-010] 퀄팅 구스다운 점퍼(BLACK) 
-----------------------------------------------------------------------
[PKG-211118-6G] 최경화  (01053251840)
ㄴ https://office.alloff.co/orders/64XYT
   [INV 211118-018] 후드 폭스 퍼 케이프 코트(베이지) 
-----------------------------------------------------------------------
[PKG-211118-6L] 이정원 (01046416708)
ㄴ https://office.alloff.co/orders/LB4W7
   [INV 211118-012] [빈폴스포츠] 투웨이 점퍼&조끼 다운 베스트(화이트) 


/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField Package.shipped_at received a naive datetime (2021-11-18 07:37:56.458332) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField Package.shipped_at received a naive datetime (2021-11-18 07:37:56.648510) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


-----------------------------------------------------------------------
[PKG-211118-69] 남정원 (01049203824)
ㄴ https://office.alloff.co/orders/NKPME
   [INV 211118-013] [빈폴스포츠] 투웨이 점퍼&조끼 다운 베스트(블랙) 
-----------------------------------------------------------------------
[PKG-211118-XQ] 장정임 (01033025953)


/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField Package.shipped_at received a naive datetime (2021-11-18 07:37:56.835644) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField Package.shipped_at received a naive datetime (2021-11-18 07:37:57.030855) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


ㄴ https://office.alloff.co/orders/77CYA
   [INV 211118-014] [빈폴스포츠] 투웨이 점퍼&조끼 다운 베스트(블랙) 
-----------------------------------------------------------------------
[PKG-211118-4V] 신정연 (01041145749)
ㄴ https://office.alloff.co/orders/4LATL
   [INV 211118-015] [빈폴스포츠] 투웨이 점퍼&조끼 다운 베스트(블랙) 
-----------------------------------------------------------------------


In [5]:
for _, row in df.iterrows():
    package = Package.objects.get(code=row["주문번호"])
    for item in package.shipping_notice_items.all():
        item.inventory.status = InventoryStatus.SHIPPED
        item.inventory.save()
        print(f"{item.inventory.code} has been shipped!")

211118-009 has been shipped!
211118-017 has been shipped!
211118-011 has been shipped!
211117-020 has been shipped!
211118-016 has been shipped!
211118-020 has been shipped!
211118-008 has been shipped!
211118-019 has been shipped!
211118-001 has been shipped!
211118-006 has been shipped!
211118-007 has been shipped!
211118-010 has been shipped!
211118-018 has been shipped!
211118-012 has been shipped!
211118-013 has been shipped!
211118-014 has been shipped!
211118-015 has been shipped!


In [7]:
for i in Inventory.objects.filter(out_order_id__in=[
    "618bdd3d26685ff05175601e",
    "617a9edb17fc2e5a1f9eabe7",
    "618b6a9f26685ff051755f2e",
    "61862bf826685ff0517556c7",
    "6183b47226685ff0517551b1",
    "618a4ca126685ff051755d58",
    "6180d93017fc2e5a1f9eb4ee",
    "618659d826685ff051755731",
    "618265fc26685ff051754f33",
    "6180619517fc2e5a1f9eb42f",
    "6182698926685ff051754f47",
]):
    order = Order.get(i.out_order_id)
    print(f"""{order.code} ({order.id}) - {i.product_name} has been shipped!""")
    i.status = InventoryStatus.SHIPPED
    i.save()

ORD-WAHUH (618bdd3d26685ff05175601e) - 캐시미어 니트 롱 드레스 has been shipped!
ORD-LWH3Y (617a9edb17fc2e5a1f9eabe7) - [택 제거 상품] Teca has been shipped!
ORD-KRJUB (6180619517fc2e5a1f9eb42f) - Torto sweatshirt(네이비) has been shipped!
ORD-X8K9D (6180d93017fc2e5a1f9eb4ee) - [로렌 랄프로렌] 브로드클로스 블루종 슬리브 셔츠 has been shipped!
ORD-PAKK6 (6183b47226685ff0517551b1) - 네오플랜 블록 구스다운 점퍼(DARK GRAY)  has been shipped!
ORD-GKYKH (61862bf826685ff0517556c7) - Ashurst quilted jacket(블랙) has been shipped!
ORD-GL7NK (618265fc26685ff051754f33) - 핸드메이드 캐시미어 코트  has been shipped!
ORD-VERXH (6182698926685ff051754f47) - 핸드메이드 캐시미어 코트  has been shipped!
ORD-BGH46 (618659d826685ff051755731) - [빈폴스포츠] 투웨이 점퍼&조끼 다운 베스트(화이트)  has been shipped!
ORD-9YUM9 (618a4ca126685ff051755d58) - 벨티드 구스 패딩(화이트 핑크)  has been shipped!
ORD-8XRFR (618b6a9f26685ff051755f2e) - 프레이드 트위드 드레스(다크네이비) has been shipped!


True